In [1]:
__depends__=[]
__dest__="../results/table_a1.tex"

# Make Table Comparing HYDRAD and EBTEL Results
Here, we'll build a table comparing the accuracy of EBTEL for various inputs as measured by HYDRAD. These HYDRAD results come from several different papers (including this one) and so are hardcoded. 

In [2]:
import sys
import os
import subprocess

import numpy as np
import astropy.constants as ac
from astropy.table import Table,Column
from astropy.io import ascii

sys.path.append(os.path.join(os.environ['EXP_DIR'],'ebtelPlusPlus','rsp_toolkit','python'))
from xml_io import InputHandler,OutputHandler

The different cases that we consider are:

* $\tau=200$ s from current paper
* $\tau=500$ s from current paper
* Case 1 from [Cargill et al. (2012a)](http://adsabs.harvard.edu/abs/2012ApJ...752..161C)
* Case 2 from [Cargill et al. (2012a)](http://adsabs.harvard.edu/abs/2012ApJ...752..161C)
* Case 3 from [Cargill et al. (2012a)](http://adsabs.harvard.edu/abs/2012ApJ...752..161C)
* Case 4 from [Cargill et al. (2012a)](http://adsabs.harvard.edu/abs/2012ApJ...752..161C)
* Group 6 from [Bradshaw and Cargill (2013)](http://adsabs.harvard.edu/abs/2013ApJ...770...12B) 
* Group 11 from [Bradshaw and Cargill (2013)](http://adsabs.harvard.edu/abs/2013ApJ...770...12B)

First, configure the base input dictionary.

In [3]:
ih = InputHandler(os.path.join(os.environ['EXP_DIR'],'ebtelPlusPlus','config','ebtel.example.cfg.xml'))
config_dict = ih.lookup_vars()

In [4]:
config_dict['calculate_dem'] = False
config_dict['use_flux_limiting'] = True
config_dict['use_adaptive_solver'] = True
config_dict['heating']['partition'] = 0.5
config_dict['force_single_fluid'] = True
config_dict['rka_error'] = 1e-9
config_dict['saturation_limit'] = 1.0
config_dict['output_filename'] = '../results/_tmp_'
config_dict['c1_rad0'] = 0.6
config_dict['use_c1_grav_correction'] = True
config_dict['use_c1_loss_correction'] = True
config_dict['tau'] = 0.1

Next, configure all the case we're going to look at.

In [5]:
appendix_a_table_cases = [
    {'hydrad_nmax':37.6,'total_time':5e+3,'loop_length':40.0,'t_pulse_half':100,'h_nano':0.1,'h_back':3.5e-5},
    {'hydrad_nmax':37.7,'total_time':5e+3,'loop_length':40.0,'t_pulse_half':250,'h_nano':0.04,'h_back':3.5e-5},
    {'hydrad_nmax':3.7,'total_time':1e+4,'loop_length':75.0,'h_nano':1.5e-3,'t_pulse_half':250.0,'h_back':2.95e-6},
    {'hydrad_nmax':10.7,'total_time':6e+3,'loop_length':25.0,'h_nano':1e-2,'t_pulse_half':100.0,'h_back':3.19e-5},
    {'hydrad_nmax':339,'total_time':2e+3,'loop_length':25.0,'h_nano':2.0,'t_pulse_half':100.0,'h_back':1.29e-3},
    {'hydrad_nmax':15.5,'total_time':2e+3,'loop_length':25.0,'h_nano':1e-2,'t_pulse_half':100.0,'h_back':4.45e-4},
    {'hydrad_nmax':350.0,'total_time':2e3,'loop_length':20.0,'h_nano':0.8,'t_pulse_half':300.0,'h_back':3.19e-5},
    {'hydrad_nmax':10.0,'total_time':5e3,'loop_length':80.0,'h_nano':0.005,'t_pulse_half':300.0,'h_back':3.19e-5}
]

Now, do all of the runs and build the table.

In [6]:
table_data = []
oh = OutputHandler(config_dict['output_filename']+'.xml',config_dict)
for t_case in appendix_a_table_cases:
    table_row = []
    #configure main options
    oh.output_dict['total_time'] = t_case['total_time']
    oh.output_dict['loop_length'] = t_case['loop_length']*1e+8
    oh.output_dict['heating']['background'] = t_case['h_back']
    oh.output_dict['heating']['events'] = [
        {'event':{'magnitude':t_case['h_nano'],
                  'rise_start':0.0,'rise_end':t_case['t_pulse_half'],
                  'decay_start':t_case['t_pulse_half'],'decay_end':2*t_case['t_pulse_half']
                 }
        }
    ]
    #run w/o correction
    oh.output_dict['c1_cond0'] = 2.0
    oh.print_to_xml()
    subprocess.call([os.path.join(os.environ['EXP_DIR'],'ebtelPlusPlus','bin','ebtel++.run'),
                     '-c',oh.output_filename])    
    nmax_uc = np.max(np.loadtxt(oh.output_dict['output_filename'])[:,3])/1e+8
    #run w/ correction
    oh.output_dict['c1_cond0'] = 6.0
    oh.print_to_xml()
    subprocess.call([os.path.join(os.environ['EXP_DIR'],'ebtelPlusPlus','bin','ebtel++.run'),
                     '-c',oh.output_filename])    
    nmax_c = np.max(np.loadtxt(oh.output_dict['output_filename'])[:,3])/1e+8
    #set table entries
    table_row = [
        2*t_case['loop_length'],
        2*t_case['t_pulse_half'],
        t_case['h_nano'],
        t_case['hydrad_nmax'],
        nmax_uc,
        nmax_c
    ]
    #save to table
    table_data.append(table_row)

Set the headers, units, formatting rules, and caption for the table.

In [7]:
headers = ['$2L$','$\\tau$','$H_0$',
           '$n_{max}$, HYDRAD','$n_{max}$, EBTEL',
           '$n_{max}$, EBTEL (Eq. \\ref{eq:c1_mod})']
units = ['(Mm)','(s)','(erg cm$^{-3}$ s$^{-1}$)','($10^8$ cm$^{-3}$)','($10^8$ cm$^{-3}$)','($10^8$ cm$^{-3}$)']
formats={}
for h,i in zip(headers,range(len(headers))):
    if i==2:
        formats[h] = '%.2g'
    elif i<2:
        formats[h] = '%.0f'
    else:
        formats[h] = '%.1f'

In [8]:
caption = r'''Comparison between HYDRAD and EBTEL with $c_1=2$ and $c_1$ given by \autoref{eq:c1_mod}, for $n<n_{eq}$.
The first three columns show the full loop length, heating pulse duration, and maximum heating rate.
The last three columns show $n_{max}$ for the three models.
Only $n_{max}$ is shown as $T_{max}$ is relatively insensitive to the value of $c_1$. 
The first two rows correspond to the $\tau=200,500$ s cases considered in this paper. 
The next four rows are the four cases shown in Table 2 of \citet{cargill_enthalpy-based_2012}. 
The last two rows are cases 6 and 11 from Table 1 of \citet{bradshaw_influence_2013}.
'''

Finally, format our data structure as an astropy table.

In [9]:
col_table = []
for i in range(len(headers)):
    temp_col = [t[i] for t in table_data]
    col_table.append(temp_col)
aas_table = Table(col_table,names=headers)
aas_table

$2L$,$\tau$,$H_0$,"$n_{max}$, HYDRAD","$n_{max}$, EBTEL","$n_{max}$, EBTEL (Eq. \ref{eq:c1_mod})"
float64,float64,float64,float64,float64,float64
80.0,200.0,0.1,37.6,45.1261,40.5023
80.0,500.0,0.04,37.7,44.9067,40.058
150.0,500.0,0.0015,3.7,3.91146,3.51059
50.0,200.0,0.01,10.7,11.534,10.3231
50.0,200.0,2.0,339.0,398.621,357.074
50.0,200.0,0.01,15.5,16.5214,14.4815
40.0,600.0,0.8,350.0,459.433,395.765
160.0,600.0,0.005,10.0,10.4359,9.30094


And print it to a file as an AAS$\TeX$ `deluxetable` object.

In [10]:
ascii.write(aas_table,output=__dest__,format='aastex',formats=formats,caption=caption,
            latexdict={'units':{h:u for h,u in zip(headers,units)},'tablefoot':r'\label{tab:table_c1_compare}'})